# __Time to slice and dice__

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

## Работа с данными

In [ ]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
# wget — утилита для скачивания файлов из интернета.
!unzip drugsCom_raw.zip

`!` — выполняет команду в Jupyter Notebook или Google Colab.  
`gzip` — утилита для сжатия и разжатия файлов в формате .gz.  
`-d (или --decompress)` — разархивирует (распаковывает) файл .gz, создавая несжатый файл.  
`-k (или --keep)` — сохраняет оригинальный сжатый файл (.gz) после распаковки.  
`-v (или --verbose)` — выводит информацию о процессе разархивации.  
`SQuAD_it-*.json.gz` — шаблон имени файла (использует * для подстановки), означающий "все файлы, начинающиеся с SQuAD_it- и заканчивающиеся на .json.gz".  
__Что делает команда:__  
Она распаковывает все файлы формата SQuAD_it-*.json.gz, оставляя как сжатую, так и несжатую версию (.json.gz и .json).

In [3]:
from datasets import load_dataset

data_files = {"train": "drugsComTrain_raw.tsv", "test": "drugsComTest_raw.tsv"}
# \t is the tab character in Python
drug_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
drug_sample = drug_dataset["train"].shuffle(seed=42).select(range(1000))
# Print the first few examples of the dataset
drug_sample[:3]

In [5]:
for split in drug_dataset.keys():
    assert len(drug_dataset[split]) == len(drug_dataset[split].unique("Unnamed: 0"))

`data_files` — переменная, содержащая словарь (dict).  
Ключи словаря:  
`"train"` — ключ, обозначающий обучающий набор данных.  
`"test"` — ключ, обозначающий тестовый набор данных.  

Цикл `for split in drug_dataset.keys()`: перебирает каждый раздел (split), например "train", "test".  
`assert len(drug_dataset[split]) == len(drug_dataset[split].unique("Unnamed: 0"))`  
- `drug_dataset[split]` — обращение к конкретному разделу датасета (например, drug_dataset["train"]).  
- `len(drug_dataset[split])` — общее количество записей (строк) в разделе.  
- `drug_dataset[split].unique("Unnamed: 0")` — вызывает метод библиотеки Hugging Face Datasets или pandas, который удаляет дубликаты в столбце "Unnamed: 0" и возвращает уникальные значения.  
- `len(drug_dataset[split].unique("Unnamed: 0"))` — количество уникальных значений в "Unnamed: 0".  
- `assert` - проверяет, что общее количество записей совпадает с количеством уникальных записей в "Unnamed: 0", то есть, что нет дубликатов в этом столбце.  
__Итог:__  
Этот код проверяет, что в каждом разделе датасета (train, test) нет дубликатов в столбце "Unnamed: 0".  
Если дубликаты есть, assert вызовет ошибку и программа остановится.

In [6]:
drug_dataset = drug_dataset.rename_column(
    original_column_name="Unnamed: 0", new_column_name="patient_id"
)
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [7]:
def lowercase_condition(example):
    return {"condition": example["condition"].lower()}

In [ ]:
drug_dataset.map(lowercase_condition)

Ошибка -  example["condition"] иногда встречается None, и Python не может вызвать .lower() для None, надо их удалить

In [ ]:
# Вариант решения
def filter_nones(x):
    return x["condition"] is not None

In [9]:
(lambda x: x * x)(3)

9

In [10]:
(lambda base, height: 0.5 * base * height)(4, 8)

16.0

Исправляем ошибку - удаляем None

In [11]:
# Но так лучше
drug_dataset = drug_dataset.filter(lambda x: x["condition"] is not None)

Filter:   0%|          | 0/161297 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53766 [00:00<?, ? examples/s]

А теперь - переводим слова в нижний регистр

In [12]:
drug_dataset = drug_dataset.map(lowercase_condition)
# Check that lowercasing worked
drug_dataset["train"]["condition"][:3]

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

['left ventricular dysfunction', 'adhd', 'birth control']

Всякий раз, когда вы имеете дело с отзывами клиентов, хорошей практикой является проверка количества слов в каждом отзыве. Отзыв может состоять всего из одного слова, например «Отлично!», или из полноценного эссе с тысячами слов, и в зависимости от варианта использования вам нужно будет обрабатывать эти крайности по-разному. Чтобы вычислить количество слов в каждом отзыве, мы воспользуемся грубой эвристикой, основанной на разделении каждого текста пробелами.

Давайте определим простую функцию, которая подсчитывает количество слов в каждом отзыве:

In [13]:
def compute_review_length(example):
    return {"review_length": len(example["review"].split())}

`compute_review_length()` возвращает словарь, ключ которого не соответствует ни одному из имен столбцов в наборе данных. В этом случае, когда compute_review_length() передается в Dataset.map(), он будет применен ко всем строкам в наборе данных __для создания нового столбца review_length:__

In [14]:
drug_dataset = drug_dataset.map(compute_review_length)
# Inspect the first training example
drug_dataset["train"][0]

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

{'patient_id': 206461,
 'drugName': 'Valsartan',
 'condition': 'left ventricular dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27,
 'review_length': 17}

In [15]:
drug_dataset["train"].sort("review_length")[:3]

{'patient_id': [111469, 13653, 53602],
 'drugName': ['Ledipasvir / sofosbuvir',
  'Amphetamine / dextroamphetamine',
  'Alesse'],
 'condition': ['hepatitis c', 'adhd', 'birth control'],
 'review': ['"Headache"', '"Great"', '"Awesome"'],
 'rating': [10.0, 10.0, 10.0],
 'date': ['February 3, 2015', 'October 20, 2009', 'November 23, 2015'],
 'usefulCount': [41, 3, 0],
 'review_length': [1, 1, 1]}

По одному слову - не информативно!  
Поэтому оставим только тот текст, где текст содержит более 30 слов

In [16]:
drug_dataset = drug_dataset.filter(lambda x: x["review_length"] > 30)
print(drug_dataset.num_rows)

Filter:   0%|          | 0/160398 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53471 [00:00<?, ? examples/s]

{'train': 138514, 'test': 46108}


Часть строк исчезло

Последнее, с чем нам нужно разобраться, это наличие кодов символов HTML в наших обзорах. Мы можем использовать модуль html Python, чтобы раскодировать эти символы, например так:

In [17]:
import html

text = "I&#039;m a transformer called BERT"
html.unescape(text)

"I'm a transformer called BERT"

In [18]:
drug_dataset = drug_dataset.map(lambda x: {"review": html.unescape(x["review"])})

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

Это работает быстрее

In [19]:
new_drug_dataset = drug_dataset.map(
    lambda x: {"review": [html.unescape(o) for o in x["review"]]}, batched=True
)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

кстати, `sorted_dataset = dataset["train"].sort("id", reverse=True)`  
сортировка в обратном порядке

Можно сортировать сразу по нескольким колонкам, передав список:  
`sorted_dataset = dataset["train"].sort(["column1", "column2"])`

чтобы токенизировать все обзоры лекарств с помощью быстрого токенизатора, мы могли бы использовать такую ​​функцию:

In [20]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [21]:
def tokenize_function(examples):
    return tokenizer(examples["review"], truncation=True)

In [22]:
%time tokenized_dataset = drug_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

CPU times: user 2min 6s, sys: 1.14 s, total: 2min 7s
Wall time: 1min 43s


In [ ]:
slow_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=False)
def slow_tokenize_function(examples):
    return slow_tokenizer(examples["review"], truncation=True)
tokenized_dataset = drug_dataset.map(slow_tokenize_function, batched=True, num_proc=8)

```
Options	            Fast tokenizer	Slow tokenizer
batched=True	      10.8s	          4min41s
batched=False	      59.2s	          5min3s
batched=True,       num_proc=8	    6.52s	41.3s
batched=False,      num_proc=8	    9.49s	45.2s
````

💡 В машинном обучении пример обычно определяется как набор признаков, которые мы передаем в модель. В некоторых контекстах эти признаки будут набором столбцов в наборе данных, но в других (например, здесь и для ответа на вопрос) несколько признаков могут быть извлечены из одного примера и принадлежать одному столбцу.

Давайте посмотрим, как это работает! Здесь мы токенизируем наши примеры и урежем их до максимальной длины 128, но мы попросим токенизатор вернуть все фрагменты текстов, а не только первый. Это можно сделать с помощью `return_overflowing_tokens=True:`

In [23]:
def tokenize_and_split(examples):
    return tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )

In [24]:
result = tokenize_and_split(drug_dataset["train"][0])
[len(inp) for inp in result["input_ids"]]

[128, 49]

 в этом коде `"input_ids"` — это токенизированное представление текста, то есть список числовых идентификаторов (ID), соответствующих токенам исходного текста "review" после обработки токенайзером.

`return_overflowing_tokens=True` — если текст длиннее max_length=128, он разбивается на несколько частей (батчей).  
Выходные данные — это словарь с ключами, такими как:  
`"input_ids"` — токенизированные ID слов.  
`"attention_mask"` — маска внимания (указывает, какие токены реальные, а какие паддинговые).  

In [ ]:
tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)
# Здесьмы получаем ошибку - есть 1000 примеров в drug_dataset["train"],
# но после токенизации с разбиением длинные тексты превращаются в несколько фрагментов, и тогда
# Теперь "input_ids" содержит 1463 строк вместо 1000, и Hugging Face Datasets не может корректно обработать разницу в длинах.

1 вариант решения проблемы.  
Мы можем удалить исходные колонки

In [26]:
tokenized_dataset = drug_dataset.map(
    tokenize_and_split, batched=True, remove_columns=drug_dataset["train"].column_names
)
# remove_columns=drug_dataset["train"].column_names - Удаляет исходные колонки (например, "review"), которые больше не нужны после токенизации.

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [27]:
len(tokenized_dataset["train"]), len(drug_dataset["train"])

(206772, 138514)

2 Вариант решения проблемы. Мы упомянули, что мы также можем справиться с проблемой несоответствия длины, сделав старые столбцы того же размера, что и новые. Для этого нам понадобится поле overflow_to_sample_mapping, которое возвращает токенизатор, когда мы устанавливаем return_overflowing_tokens=True. Оно дает нам сопоставление нового индекса признаков с индексом образца, из которого он произошел. Используя это, мы можем связать каждый ключ, присутствующий в нашем исходном наборе данных, со списком значений нужного размера, повторяя значения каждого примера столько раз, сколько он генерирует новых признаков:

In [28]:
def tokenize_and_split(examples):
    result = tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )
    # Extract mapping between new and old indices
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result

In [29]:
tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)
tokenized_dataset

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 206772
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 68876
    })
})

## From Dataset s to DataFrame s and back

Чтобы включить преобразование между различными сторонними библиотеками, 🤗 Datasets предоставляет функцию Dataset.set_format(). Эта функция изменяет только выходной формат набора данных, поэтому вы можете легко переключиться на другой формат, не затрагивая базовый формат данных, которым является Apache Arrow. Форматирование выполняется на месте. Для демонстрации давайте преобразуем наш набор данных в Pandas:

In [30]:
drug_dataset.set_format("pandas")

In [31]:
drug_dataset["train"][:3]

,patient_id,drugName,condition,review,rating,date,usefulCount,review_length
0,95260,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,141
1,92703,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,134
2,138000,Ortho Evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,89


Давайте создадим pandas.DataFrame для всего обучающего набора, выбрав все элементы drug_dataset["train"]:

In [32]:
train_df = drug_dataset["train"][:]

🚨 Под капотом Dataset.set_format() изменяет формат возврата для метода __getitem__() набора данных. Это означает, что когда мы хотим создать новый объект, например train_df, из набора данных в формате "pandas", нам нужно разрезать весь набор данных, чтобы получить pandas.DataFrame. Вы можете убедиться сами, что тип drug_dataset["train"] — это Dataset, независимо от формата вывода.

Отсюда мы можем использовать все необходимые нам функциональные возможности Pandas. Например, мы можем выполнить причудливую цепочку для вычисления распределения классов среди записей условий:

In [40]:
frequencies = (
    train_df["condition"] # 1️⃣ Берём колонку "condition"
    .value_counts()  # 2️⃣ Считаем, сколько раз встречается каждое значение и при этом создаётся Series с именем "count"
    .to_frame() # 3️⃣ Превращаем результат в DataFrame .reset_index(), столбец остаётся "count"
    .reset_index() #4️⃣ Переносим индексы в новую колонку
    .rename(columns={"index": "condition",  "count": "frequency"}) # 5️⃣ Переименовываем колонки
)
frequencies.head()

,condition,frequency
0,birth control,27655
1,depression,8023
2,acne,5209
3,anxiety,4991
4,pain,4744


Этот код считает частоту встречаемости каждого уникального значения в колонке "condition" внутри train_df и сохраняет результат в новый DataFrame frequencies.

 Пример работы
✅ Допустим, у нас есть DataFrame train_df:   
`condition`  
Headache  
Flu  
Headache  
Allergy  
Flu  
Flu  
✅ Шаг 1. value_counts() — считаем частоту значений  
`train_df["condition"].value_counts()`  
🔹 Результат:  
```
Flu         3  
Headache    2  
Allergy     1  
```
✅ Шаг 2. .to_frame() — превращаем в DataFrame   
`train_df["condition"].value_counts().to_frame()`  
🔹 Теперь это таблица:  
`condition`  
3  
2  
1  
Но колонка называется "condition", что может быть путаницей.  
✅ Шаг 3. .reset_index() — превращаем индексы в обычный столбец
`train_df["condition"].value_counts().to_frame().reset_index()`  
🔹 Теперь у нас таблица:
```
index	    condition  
Flu	      3  
Headache	2  
Allergy	  1  
```
✅ Шаг 4.   
`.rename(columns={"index": "condition", "condition": "frequency"})`  
🔹 Финальный вид frequencies:  

```
condition	frequency
Flu	      3
Headache	2
Allergy	  1
```
🎯 Вывод  
Этот код:  
Считает частоту встречаемости каждой болезни ("condition") в датасете.  
Создаёт новый DataFrame с колонками "condition" и "frequency".  

И как только мы закончим анализ Pandas, мы всегда можем создать новый объект Dataset, используя функцию Dataset.from_pandas() следующим образом:

In [36]:
from datasets import Dataset

freq_dataset = Dataset.from_pandas(frequencies)
freq_dataset

Dataset({
    features: ['frequency', 'count'],
    num_rows: 819
})

✏️ Попробуйте! Вычислите средний рейтинг для каждого препарата и сохраните результат в новом наборе данных.

На этом мы завершаем наш обзор различных методов предварительной обработки, доступных в 🤗 наборах данных. Чтобы завершить раздел, давайте создадим набор проверки, чтобы подготовить набор данных для обучения классификатора. Перед этим мы сбросим формат выходных данных drug_dataset с «pandas» на «arrow»:

In [41]:
drug_dataset.reset_format()

In [ ]:
drug_dataset["train"][0]

## Создание набора для проверки


Хотя у нас есть тестовый набор, который мы могли бы использовать для оценки, хорошей практикой является оставить тестовый набор нетронутым и создать отдельный набор для проверки во время разработки. Как только вы будете довольны производительностью своих моделей на наборе для проверки, вы можете провести окончательную проверку работоспособности тестового набора. Этот процесс помогает снизить риск того, что вы переобучитесь тестовому набору и развернете модель, которая не сработает на реальных данных.

🤗 Datasets предоставляет функцию Dataset.train_test_split(), которая основана на известной функциональности из scikit-learn. Давайте используем ее, чтобы разделить наш обучающий набор на обучающую и проверочную части (мы задаем аргумент seed для воспроизводимости):

In [43]:
drug_dataset_clean = drug_dataset["train"].train_test_split(train_size=0.8, seed=42)
# Rename the default "test" split to "validation"
drug_dataset_clean["validation"] = drug_dataset_clean.pop("test")
# Add the "test" set to our `DatasetDict`
drug_dataset_clean["test"] = drug_dataset["test"]
drug_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

Метод `.pop("test")` удаляет и возвращает test-часть датасета, чтобы затем сохранить её под новым именем "validation".

Отлично, теперь мы подготовили набор данных, готовый для обучения некоторых моделей! В разделе 5 мы покажем вам, как загружать наборы данных в Hugging Face Hub, но сейчас давайте завершим наш анализ, рассмотрев несколько способов сохранения наборов данных на локальном компьютере.

In [44]:
drug_dataset_clean.save_to_disk("drug-reviews")

Saving the dataset (0/1 shards):   0%|          | 0/110811 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/27703 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/46108 [00:00<?, ? examples/s]

Хотя 🤗 Datasets будет кэшировать каждый загруженный набор данных и операции, выполненные с ним, бывают случаи, когда вам нужно сохранить набор данных на диск (например, в случае удаления кэша). Как показано в таблице ниже, 🤗 Datasets предоставляет три основные функции для сохранения вашего набора данных в разных форматах:

```
Data format	      Function
Arrow	            Dataset.save_to_disk()
CSV	              Dataset.to_csv()
JSON	            Dataset.to_json()
```

This will create a directory with the following structure:
```
drug-reviews/
├── dataset_dict.json
├── test
│   ├── dataset.arrow
│   ├── dataset_info.json
│   └── state.json
├── train
│   ├── dataset.arrow
│   ├── dataset_info.json
│   ├── indices.arrow
│   └── state.json
└── validation
    ├── dataset.arrow
    ├── dataset_info.json
    ├── indices.arrow
    └── state.json
```

где мы видим, что каждое разделение связано со своей собственной таблицей dataset.arrow и некоторыми метаданными в dataset_info.json и state.json. Вы можете представить себе формат Arrow как замысловатую таблицу столбцов и строк, оптимизированную для создания высокопроизводительных приложений, обрабатывающих и передающих большие наборы данных.

После сохранения набора данных мы можем загрузить его с помощью функции load_from_disk() следующим образом:

In [45]:
from datasets import load_from_disk

drug_dataset_reloaded = load_from_disk("drug-reviews")
drug_dataset_reloaded

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

Для форматов CSV и JSON нам нужно хранить каждое разделение как отдельный файл. Один из способов сделать это — перебрать ключи и значения в объекте DatasetDict:

In [46]:
for split, dataset in drug_dataset_clean.items():
    dataset.to_json(f"drug-reviews-{split}.jsonl")

Creating json from Arrow format:   0%|          | 0/111 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/28 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/47 [00:00<?, ?ba/s]

Этот цикл перебирает все части (например, split="train" и dataset=Dataset(...)).  
`dataset.to_json(f"drug-reviews-{split}.jsonl")` - сохраняет каждый сплит в .jsonl файл  
🔹 `.to_json("имя_файла.jsonl")` — сохраняет датасет в формате JSON Lines (.jsonl).  
🔹 `f"drug-reviews-{split}.jsonl"` — формирует имя файла:  

Для train → "drug-reviews-train.jsonl"
Для validation → "drug-reviews-validation.jsonl"
...


Он сохраняет каждое разделение в формате JSON Lines, где каждая строка в наборе данных хранится как одна строка JSON. Вот как выглядит первый пример:

In [47]:
!head -n 1 drug-reviews-train.jsonl

{"patient_id":89879,"drugName":"Cyclosporine","condition":"keratoconjunctivitis sicca","review":"\"I have used Restasis for about a year now and have seen almost no progress.  For most of my life I've had red and bothersome eyes. After trying various eye drops, my doctor recommended Restasis.  He said it typically takes 3 to 6 months for it to really kick in but it never did kick in.  When I put the drops in it burns my eyes for the first 30 - 40 minutes.  I've talked with my doctor about this and he said it is normal but should go away after some time, but it hasn't. Every year around spring time my eyes get terrible irritated  and this year has been the same (maybe even worse than other years) even though I've been using Restasis for a year now. The only difference I notice was for the first couple weeks, but now I'm ready to move on.\"","rating":2.0,"date":"April 20, 2013","usefulCount":69,"review_length":147}


Затем мы можем использовать методы из раздела 2 для загрузки файлов JSON следующим образом:

In [48]:
data_files = {
    "train": "drug-reviews-train.jsonl",
    "validation": "drug-reviews-validation.jsonl",
    "test": "drug-reviews-test.jsonl",
}
drug_dataset_reloaded = load_dataset("json", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [49]:
drug_dataset_reloaded

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

Вот и все, что касается обработки данных с помощью 🤗 Datasets! Теперь, когда у нас есть очищенный набор данных для обучения модели, вот несколько идей, которые вы можете попробовать:

Используйте методы из Главы 3 для обучения классификатора, который может предсказывать состояние пациента на основе обзора препарата.
Используйте конвейер резюмирования из Главы 1 для создания резюме обзоров.
Далее мы рассмотрим, как 🤗 Datasets могут позволить вам работать с огромными наборами данных, не взрывая ваш ноутбук!